# Causal Evaluation of Probes

This is a simple tutorial showing you to collect activations from intervention-points in a model. We'll compare 1D DAS IIA on each layer and position for `block_output` in pythia-70M with logistic regression probing accuracy. The task we'll look at is gender prediction, where gendered names are used in templates like "[name] walked because", which elicits the associated gendered pronoun "he" or "she" as the next-token prediction for this model.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/frankaging/pyvene/blob/main/tutorials/advance_tutorials/Probing_Gender.ipynb)


In [144]:
__author__ = "Aryaman Arora"
__version__ = "01/10/2024"

## Setup

In [1]:
try:
    # This library is our indicator that the required installs
    # need to be done.
    import pyvene as pv

except ModuleNotFoundError:
    !pip install git+https://github.com/stanfordnlp/pyvene.git

nnsight is not detected. Please install via 'pip install nnsight' for nnsight backend.


In [2]:
import pandas as pd
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
)
import torch
import random
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

%config InlineBackend.figure_formats = ['svg']
from plotnine import (
    ggplot,
    geom_tile,
    aes,
    facet_wrap,
    theme,
    element_text,
    geom_bar,
    geom_hline,
    scale_y_log10,
    geom_line,
    geom_point,
    geom_text,
    ggtitle, xlab, ylab, 
    ggsave
)
from plotnine.scales import scale_y_reverse, scale_fill_cmap
from tqdm import tqdm
from collections import namedtuple

## Load model and data

In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = "EleutherAI/pythia-410m" # "EleutherAI/pythia-6.9B"
tokenizer = AutoTokenizer.from_pretrained(model)
tokenizer.pad_token = tokenizer.eos_token
gpt = AutoModelForCausalLM.from_pretrained(
    model,
    revision="main",
    torch_dtype=torch.bfloat16 if model == "EleutherAI/pythia-6.9b" else torch.float32,
).to(device)

We have a list of 100 names for each gender, and we'll filter for names that are one token in length. We'll further filter for examples the model agrees with our labels for, since some of these names might be ambiguous or the model might not have the expected behaviour. This ensures that baseline IIA is 0.

In [4]:
from datasets import load_dataset

ds = load_dataset("tasksource/babi_nli", "basic-induction")


In [6]:
# Filtrujemy dane, gdzie label != 0
filtered_ds = ds.filter(lambda example: example['label'] == 1)
# Tworzymy listę z kolumny 'sentence' (przekształcamy ją w 'premise')
sentences = filtered_ds['train']['premise']
hypotheses = filtered_ds['train']['hypothesis']

In [6]:
hypotheses

['Greg is yellow.',
 'Brian is white.',
 'Lily is green.',
 'Lily is green.',
 'Julius is yellow.',
 'Brian is yellow.',
 'Brian is green.',
 'Julius is yellow.',
 'Greg is green.',
 'Julius is gray.',
 'Bernhard is green.',
 'Greg is gray.',
 'Bernhard is white.',
 'Brian is green.',
 'Greg is yellow.',
 'Lily is green.',
 'Lily is white.',
 'Greg is yellow.',
 'Greg is green.',
 'Brian is white.',
 'Lily is white.',
 'Brian is yellow.',
 'Brian is green.',
 'Bernhard is yellow.',
 'Brian is gray.',
 'Lily is white.',
 'Brian is white.',
 'Greg is green.',
 'Greg is white.',
 'Brian is green.',
 'Greg is white.',
 'Lily is white.',
 'Lily is white.',
 'Brian is green.',
 'Lily is white.',
 'Bernhard is gray.',
 'Julius is white.',
 'Brian is yellow.',
 'Julius is gray.',
 'Greg is green.',
 'Lily is yellow.',
 'Greg is green.',
 'Bernhard is green.',
 'Julius is white.',
 'Julius is yellow.',
 'Greg is yellow.',
 'Julius is white.',
 'Lily is white.',
 'Brian is green.',
 'Lily is gre

In [7]:
# def sort_sentences_by_hypothesis(sentences, hypotheses):
#     result = []

#     for sentence, hypothesis in zip(sentences, hypotheses):
#         # Dzielimy hipotezę na słowa i ekstraktujemy etykietę
#         hypothesis_words = hypothesis.split()
#         label = hypothesis_words[-1]  # Etykieta to ostatnie słowo
#         hypothesis_without_label = ' '.join(hypothesis_words[:-1])  # Hipoteza bez etykiety

#         first_word = hypothesis_words[0]  # Pierwsze słowo hipotezy
        
#         # Dzielimy zdanie na części (rozdzielając je kropkami)
#         sentence_parts = sentence.split('. ')
#         sentence_parts = [part.rstrip('.') for part in sentence_parts]  # Usuwamy kropki na końcu
        
#         # Zaczynamy od części, która zawiera pierwsze słowo z hipotezy
#         first_word_sentence = [part for part in sentence_parts if first_word.lower() in part.lower()]
        
#         sorted_sentence_parts = []
        
#         if first_word_sentence:
#             # Dodajemy część zawierającą pierwsze słowo na początek
#             sorted_sentence_parts.append(first_word_sentence[0])
        
#         # Pozostałe części zdania
#         other_parts = [part for part in sentence_parts if part not in first_word_sentence]
#         sorted_sentence_parts.extend(other_parts)

#         # Łączymy posortowane części w jedno zdanie
#         sorted_sentence = '. '.join(sorted_sentence_parts).strip()
        
#         # Zapewniamy, że zdanie kończy się kropką
#         if sorted_sentence and not sorted_sentence.endswith('.'):
#             sorted_sentence += '.' 
#         sentence_words = sorted_sentence.split(' ')
#         sentence_words[0] = "Greg"  # Zamiana pierwszego słowa na "greg"
#         sorted_sentence = ' '.join(sentence_words) 

#         # Dodajemy posortowane zdanie i hipotezę bez etykiety do wyniku
#         result.append((sorted_sentence + " " + hypothesis_without_label, label))
    
#     return result

# sorted_sentences = sort_sentences_by_hypothesis(sentences, hypotheses)

# for sentence, label in sorted_sentences:
#     print("Zdanie:", sentence)
#     print("Label:", label)

In [21]:
# def sort_sentences_by_hypothesis(sentences, hypotheses):
#     result = []

#     for sentence, hypothesis in zip(sentences, hypotheses):
#         # Dzielimy hipotezę na słowa i ekstraktujemy etykietę
#         hypothesis_words = hypothesis.split()
#         label = hypothesis_words[-1]  # Etykieta to ostatnie słowo
#         hypothesis_without_label = ' '.join(hypothesis_words[:-1])  # Hipoteza bez etykiety

#         # Filtrujemy tylko te zdania, które mają "greg" w hipotezie
#         if "greg" not in hypothesis.lower():
#             continue
        
#         # Dzielimy zdanie na części (rozdzielając je kropkami)
#         sentence_parts = sentence.split('. ')
#         sentence_parts = [part.rstrip('.') for part in sentence_parts]  # Usuwamy kropki na końcu
        
#         # Inicjujemy puste listy do przechowywania wyników
#         greg_animal = []
#         greg_color = []
#         lily_animal = []
#         lily_color = []
#         brian_animal = []
#         brian_color = []
#         bernhard_animal = []
#         bernhard_color = []
#         julius_animal = []
#         julius_color = []

#         # Przypisujemy części zdania do odpowiednich list
#         for part in sentence_parts:
#             part_lower = part.lower()

#             # Przypisanie zdań do odpowiednich kategorii
#             if "greg" in part_lower:
#                 if "is a" in part_lower:
#                     greg_animal.append(part)
#                 elif "is" in part_lower:
#                     greg_color.append(part)
#             elif "lily" in part_lower:
#                 if "is a" in part_lower:
#                     lily_animal.append(part)
#                 elif "is" in part_lower:
#                     lily_color.append(part)
#             elif "brian" in part_lower:
#                 if "is a" in part_lower:
#                     brian_animal.append(part)
#                 elif "is" in part_lower:
#                     brian_color.append(part)
#             elif "bernhard" in part_lower:
#                 if "is a" in part_lower:
#                     bernhard_animal.append(part)
#                 elif "is" in part_lower:
#                     bernhard_color.append(part)
#             elif "julius" in part_lower:
#                 if "is a" in part_lower:
#                     julius_animal.append(part)
#                 elif "is" in part_lower:
#                     julius_color.append(part)

#         # Łączymy wyniki w odpowiedniej kolejności
#         sorted_sentence = '. '.join(
#             greg_animal + greg_color + 
#             lily_animal + lily_color + 
#             brian_animal + brian_color + 
#             bernhard_animal + bernhard_color + 
#             julius_animal + julius_color
#         ).strip()

#         # Zapewniamy, że zdanie kończy się kropką
#         if sorted_sentence and not sorted_sentence.endswith('.'):
#             sorted_sentence += '.'  
#         label_without_dot = label.rstrip('.')

#         # Dodajemy posortowane zdanie i hipotezę bez etykiety do wyniku
#         result.append((sorted_sentence + " " + hypothesis_without_label, label_without_dot))
    
#     return result

# # Przykład użycia
# sorted_sentences = sort_sentences_by_hypothesis(sentences, hypotheses)

# for sentence, label in sorted_sentences:
#     print("Zdanie:", sentence)
#     print("Label:", label)
# print(len(sorted_sentences))

In [29]:
tokenizer.tokenize('green')

['green']

In [9]:
def count_tokens_in_sentences(sentences, tokenizer):
    sentence_token_counts = []

    for sentence in sentences:
        tokens = tokenizer.tokenize(sentence)  # Tokenizowanie zdania
        sentence_token_counts.append(len(tokens))  # Liczenie liczby tokenów

    return sentence_token_counts


# Wywołanie funkcji
token_counts = count_tokens_in_sentences(sorted_sentences, tokenizer)

# liczba zdań z poszczególną liczbą tokenów
token_counts_df = pd.DataFrame(token_counts, columns=["token_count"])
token_counts_df["count"] = 1
token_counts_df = token_counts_df.groupby("token_count").count().reset_index()
token_counts_df

,token_count,count
0,46,6
1,47,11
2,48,36
3,49,20
4,50,11
5,51,7


In [10]:
#filtrowanie zdań które mają 49 tokenów
filtered_pairs = [(sentence, label) for sentence, label in sorted_sentences if len(tokenizer.tokenize(sentence)) == 47]

In [11]:
len(filtered_pairs)

36

In [12]:
sentence = "<|endoftext|> Greg is a lion. Lily is a rhino. Lily is white. Brian is a lion. Brian is yellow. Bernhard is a swan. Bernhard is white. Julius is a rhino. Julius is white. Greg is"

# Tokenizacja
tokens = tokenizer.tokenize(sentence)

# Wyświetl tokeny
print(len(tokens))

49


In [13]:
random.sample(filtered_pairs,2)

[('Greg is a swan. Lily is a lion. Lily is gray. Brian is a swan. Brian is green. Bernhard is a lion. Bernhard is gray. Julius is a lion. Julius is gray. Greg is',
  'green'),
 ('Greg is a lion. Lily is a frog. Lily is yellow. Brian is a swan. Brian is gray. Bernhard is a swan. Bernhard is green. Julius is a lion. Julius is gray. Greg is',
  'gray')]

In [74]:
tokenizer.tokenize("bear")

['bear']

In [5]:
Example = namedtuple("Example", ["base", "src", "base_label", "src_label"])

colors_all = ["white", "yellow", "blue", "green", "red", "black", "brown", "orange", "pink", "grey", "gold"]


def sample_example(tokenizer):

    colors_base = random.sample(colors_all, 4)
    greg_color_base = random.choice(colors_base)

    lily_color_base = colors_base[0]
    brian_color_base = colors_base[1]
    bernhard_color_base = colors_base[2]
    julius_color_base = colors_base[3]

    if greg_color_base == lily_color_base:
        base_label = 'cat'
    elif greg_color_base == brian_color_base:
        base_label = 'wolf'
    elif greg_color_base == bernhard_color_base:
        base_label = 'bee'
    else:
        base_label = 'fox'

    colors_src = random.sample(colors_all, 4)
    greg_color_src = random.choice(colors_src)

    lily_color_src = colors_src[0]
    brian_color_src = colors_src[1]
    bernhard_color_src = colors_src[2]
    julius_color_src = colors_src[3]

    if greg_color_src == lily_color_src:
        src_label = 'cat'
    elif greg_color_src == brian_color_src:
        src_label = 'wolf'
    elif greg_color_src == bernhard_color_src:
        src_label = 'bee'
    else:
        src_label = 'fox'

    # make pair
    base = tokenizer(f"<|endoftext|> Greg is {greg_color_base}. Lily is a cat. Lily is {lily_color_base}. Brian is a wolf. Brian is {brian_color_base}. Bernhard is a bee. Bernhard is {bernhard_color_base}. Julius is a fox. Julius is {julius_color_base}. Greg is a", return_tensors="pt")
    src = tokenizer(f"<|endoftext|> Greg is {greg_color_src}. Lily is a cat. Lily is {lily_color_src}. Brian is a wolf. Brian is {brian_color_src}. Bernhard is a bee. Bernhard is {bernhard_color_src}. Julius is a fox. Julius is {julius_color_src}. Greg is a", return_tensors="pt")
    base_label = tokenizer.encode(" " + base_label)[0]
    src_label = tokenizer.encode(" " + src_label)[0]
    return Example(base, src, base_label, src_label)

In [6]:
random.sample(colors_all, 4)

['pink', 'white', 'grey', 'black']

In [7]:
sample_example(tokenizer)
len(sample_example(tokenizer).base['input_ids'][0])

46

In [8]:
def generate_n_doable_examples(n, model, tokenizer):
    examples = []
    iterator = tqdm(range(n))
    while len(examples) < n:
        ex = sample_example(tokenizer)
        for k, v in ex.base.items():
            if v is not None and isinstance(v, torch.Tensor):
                ex.base[k] = v.to(model.device)
        for k, v in ex.src.items():
            if v is not None and isinstance(v, torch.Tensor):
                ex.src[k] = v.to(model.device)
        logits_base = model(**ex.base).logits[0, -1]
        logits_src = model(**ex.src).logits[0, -1]
        if (
            logits_base[ex.base_label] > logits_base[ex.src_label]
            and logits_src[ex.src_label] > logits_src[ex.base_label]
        ):
            examples.append(ex)
            iterator.update(1)
    return examples

In [9]:
# make dataset
total_steps = 100
trainset = generate_n_doable_examples(total_steps, gpt, tokenizer)
evalset = generate_n_doable_examples(50, gpt, tokenizer)

100%|██████████| 50/50 [02:35<00:00,  3.11s/it]


In [29]:
trainset

[Example(base={'input_ids': tensor([[    0, 13916,   310, 14863,    15, 32817,   310,   247,  5798,    15,
          32817,   310,  8862,    15, 14468,   310,   247, 25872,    15, 14468,
            310,  3168,    15, 10246, 10984,   310,   247, 30747,    15, 10246,
          10984,   310, 14863,    15, 44402,   310,   247, 30013,    15, 44402,
            310, 14370,    15, 13916,   310,   247]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
        device='cuda:0')}, src={'input_ids': tensor([[    0, 13916,   310, 14370,    15, 32817,   310,   247,  5798,    15,
          32817,   310,  4797,    15, 14468,   310,   247, 25872,    15, 14468,
            310, 14370,    15, 10246, 10984,   310,   247, 30747,    15, 10246,
          10984,   310,  5328,    15, 44402,   310,   247, 30013,    15, 44402,
            310,  2502,    15, 13916,   

## DAS

This is the usual 1D DAS setup, training on batch size of 1.

In [10]:
def intervention_config(intervention_site, layer, num_dims=1):
    config = pv.IntervenableConfig([
        {
            "layer": layer,
            "component": intervention_site,
            "intervention_type": pv.LowRankRotatedSpaceIntervention,
            "low_rank_dimension": num_dims,
        }
    ])
    return config

In [11]:
# loss function
loss_fct = torch.nn.CrossEntropyLoss()

def calculate_loss(logits, label):
    """Calculate cross entropy between logits and a single target label (can be batched)"""
    shift_labels = label.to(logits.device)
    loss = loss_fct(logits, shift_labels)
    return loss

In [28]:
# intervention settings
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

stats = []
num_layers = gpt.config.num_hidden_layers

# loop over layers and positions
for layer in range(num_layers):
    for position in range(46):
        print(f"layer: {layer}, position: {position}")

        # set up intervenable model
        config = intervention_config("block_output", layer, 1)
        intervenable = pv.IntervenableModel(config, gpt)
        intervenable.set_device(device)
        intervenable.disable_model_gradients()

        # set up optimizer
        optimizer_params = []
        for k, v in intervenable.interventions.items():
            try:
                optimizer_params.append({"params": v[0].rotate_layer.parameters()})
            except:
                pass
        optimizer = torch.optim.Adam(optimizer_params, lr=1e-3)
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=int(0.1 * total_steps),
            num_training_steps=total_steps,
        )

        # training loop
        iterator = tqdm(trainset)
        for example in iterator:
            # forward pass
            _, counterfactual_outputs = intervenable(
                example.base,
                [example.src],
                {"sources->base": position},
            )

            # loss
            logits = counterfactual_outputs.logits[:, -1]
            loss = calculate_loss(logits, torch.tensor([example.src_label]).to(device))
            iterator.set_postfix({"loss": f"{loss.item():.3f}"})

            # backward
            loss.backward()
            optimizer.step()
            scheduler.step()

        # eval
        with torch.no_grad():
            iia = 0
            iterator = tqdm(evalset)
            for example in iterator:
                # forward
                _, counterfactual_outputs = intervenable(
                    example.base,
                    [example.src],
                    {"sources->base": position},
                )

                # calculate iia
                logits = counterfactual_outputs.logits[0, -1]
                if logits[example.src_label] > logits[example.base_label]:
                    iia += 1

            # stats
            iia = iia / len(evalset)
            stats.append({"layer": layer, "position": position, "iia": iia})
            print(f"iia: {iia:.3%}")
df = pd.DataFrame(stats)
df.to_csv(f"./tutorial_data/pyvene_animal_das_410m.csv")

layer: 0, position: 0


100%|██████████| 50/50 [00:03<00:00, 16.60it/s]


iia: 0.000%
layer: 0, position: 1


100%|██████████| 50/50 [00:03<00:00, 16.34it/s]


iia: 0.000%
layer: 0, position: 2


...

iia: 0.000%
layer: 23, position: 44


100%|██████████| 50/50 [00:03<00:00, 14.03it/s]


iia: 0.000%
layer: 23, position: 45


100%|██████████| 50/50 [00:03<00:00, 14.74it/s]

iia: 98.000%


And this is the plot of IIA. In layers 2 and 3 it seems the gender is represented across positions 1-3, and entirely in position 3 in later layers.

In [11]:
tokenizer.tokenize("<|endoftext|> Greg is red. Lily is a cat. Lily is red. Brian is a wolf. Brian is yellow. Bernhard is a bee. Bernhard is brown. Julius is a fox. Julius is blue. Greg is a")

['<|endoftext|>',
 'ĠGreg',
 'Ġis',
 'Ġred',
 '.',
 'ĠLily',
 'Ġis',
 'Ġa',
 'Ġcat',
 '.',
 'ĠLily',
 'Ġis',
 'Ġred',
 '.',
 'ĠBrian',
 'Ġis',
 'Ġa',
 'Ġwolf',
 '.',
 'ĠBrian',
 'Ġis',
 'Ġyellow',
 '.',
 'ĠBern',
 'hard',
 'Ġis',
 'Ġa',
 'Ġbee',
 '.',
 'ĠBern',
 'hard',
 'Ġis',
 'Ġbrown',
 '.',
 'ĠJulius',
 'Ġis',
 'Ġa',
 'Ġfox',
 '.',
 'ĠJulius',
 'Ġis',
 'Ġblue',
 '.',
 'ĠGreg',
 'Ġis',
 'Ġa']

In [29]:
df = pd.read_csv(f"./tutorial_data/pyvene_animal_das_410m.csv")
df["layer"] = df["layer"].astype(int)
df["pos"] = df["position"].astype(int)
df["IIA"] = df["iia"].astype(float)

custom_labels = ["EOS", "Greg", "is",  "<color1>",  ".",  "Lily", "is", "a",  "<animal2>",  ".",  "Lily",  "is", "<color2>", ".", "Brian",  "is",  "a",  "<animal2>",  ".", "Brian", "is", "<color3>",  ".",  "Bern", "hard",  "is", "a", "<animal3>", ".", "Bern", "hard", "is",  "<color5>",  ".", "Julius", "is", "a", "<animal4>", ".", "Julius", "is", "<color6>", ".", "Greg", "is", "a"]
breaks = range(len(custom_labels))

plot = (
    ggplot(df, aes(x="layer", y="pos"))    
    + geom_tile(aes(fill="IIA"))
    + scale_fill_cmap("Purples") + xlab("layers")
    + scale_y_reverse(
        limits = (-0.5, 52.5), 
        breaks=breaks, labels=custom_labels) 
    + theme(figure_size=(25, 20)) + ylab("") 
    + theme(axis_text_y  = element_text(angle = 0, hjust = 1))
    + ggtitle("Trained Intervention (DAS)")
)
ggsave(
    plot, filename=f"./tutorial_data/pyvene_animal_das_410m.pdf", dpi=200
)
print(plot)

c:\Users\kjani\AppData\Local\Programs\Python\Python310\lib\site-packages\plotnine\ggplot.py:615: PlotnineWarning: Saving 25 x 20 in image.
c:\Users\kjani\AppData\Local\Programs\Python\Python310\lib\site-packages\plotnine\ggplot.py:616: PlotnineWarning: Filename: ./tutorial_data/pyvene_animal_das_410m.pdf


<ggplot: (2500 x 2000)>


## Probing

We'll define a dummy intervention `CollectActivation` to collect activations and train a simple probe.

In [12]:
def probing_config(intervention_site, layer):
    """Generate intervention config."""

    # init
    config = pv.IntervenableConfig([{
        "layer": layer,
        "component": intervention_site,
        "intervention_type": pv.CollectIntervention,
    }])
    return config

This is the training loop.

In [17]:
tokenizer.decode([30013])

' fox'

In [26]:
#numeric token of word fox
tokenizer.encode("fox")

[15736]

In [13]:
# intervention settings
stats = []
num_layers = gpt.config.num_hidden_layers

label_mapping = {30747: 0, 25872: 1, 5798:2, 15736:3, 30013:3}

# loop over layers and positions
with torch.no_grad():
    for layer in range(num_layers):
        for position in range(46):
            print(f"layer: {layer}, position: {position}")

            # set up intervenable model
            config = probing_config("block_output", layer)
            intervenable = pv.IntervenableModel(config, gpt)
            intervenable.set_device(device)
            intervenable.disable_model_gradients()

            # training loop
            activations, labels = [], []
            iterator = tqdm(trainset)
            for example in iterator:
                # forward pass
                base_outputs, _ = intervenable(
                    example.base,
                    unit_locations={"base": position},
                )
                base_activations = base_outputs[1][0]

                src_outputs, _ = intervenable(
                    example.src,
                    unit_locations={"base": position},
                )
                src_activations = src_outputs[1][0]
                
                # collect activation
                activations.extend(
                    [base_activations.detach()[0].cpu().numpy(), src_activations.detach()[0].cpu().numpy()]
                )
                labels.extend([example.base_label, example.src_label])
            labels = [label_mapping[label] for label in labels]
            
            # train logistic regression
            lr = LogisticRegression(random_state=42, max_iter=1000).fit(
                activations, labels
            )

            # eval
            activations, labels = [], []
            iterator = tqdm(evalset)
            for example in iterator:
                # forward pass
                base_outputs, _ = intervenable(
                    example.base,
                    unit_locations={"base": position},
                )
                base_activations = base_outputs[1][0]

                src_outputs, _ = intervenable(
                    example.src,
                    unit_locations={"base": position},
                )
                src_activations = src_outputs[1][0]
                
                # collect activation
                activations.extend(
                    [base_activations.detach()[0].cpu().numpy(), src_activations.detach()[0].cpu().numpy()]
                )
                labels.extend([example.base_label, example.src_label])
            labels = [label_mapping[label] for label in labels]

            # stats
            acc = lr.score(activations, labels)
            f1 = f1_score(labels, lr.predict(activations), average="macro")
            stats.append({"layer": layer, "position": position, "acc": acc, "f1": f1})
            print(f"acc: {acc:.3%}, f1: {f1:.3f}")
df = pd.DataFrame(stats)
df.to_csv(f"./tutorial_data/pyvene_animal_probe_410m.csv")

layer: 0, position: 0


100%|██████████| 50/50 [00:03<00:00, 15.66it/s]


acc: 50.000%, f1: 0.222
layer: 0, position: 1


100%|██████████| 50/50 [00:03<00:00, 13.67it/s]


acc: 50.000%, f1: 0.222
layer: 0, position: 2


...

acc: 100.000%, f1: 1.000
layer: 23, position: 44


100%|██████████| 50/50 [00:03<00:00, 14.31it/s]


acc: 100.000%, f1: 1.000
layer: 23, position: 45


100%|██████████| 50/50 [00:03<00:00, 14.30it/s]

acc: 100.000%, f1: 1.000


And the probe accuracy plot is below. Note the extremely high accuracy at all positions at and after the name! Early layers at later positions are better but it saturates much before the IIA for DAS. This shows how unreliable probes are for tracing causal effect.

In [14]:
df = pd.read_csv(f"./tutorial_data/pyvene_animal_probe_410m.csv")
df["layer"] = df["layer"].astype(int)
df["pos"] = df["position"].astype(int)
df["ACC"] = df["acc"].astype(float)

custom_labels = ["EOS", "Greg", "is",  "<color1>",  ".",  "Lily", "is", "a",  "<animal2>",  ".",  "Lily",  "is", "<color2>", ".", "Brian",  "is",  "a",  "<animal2>",  ".", "Brian", "is", "<color3>",  ".",  "Bern", "hard",  "is", "a", "<animal3>", ".", "Bern", "hard", "is",  "<color5>",  ".", "Julius", "is", "a", "<animal4>", ".", "Julius", "is", "<color6>", ".", "Greg", "is", "a"]
breaks = range(len(custom_labels))

plot = (
    ggplot(df, aes(x="layer", y="pos", fill="ACC"))    
    + geom_tile()
    + scale_fill_cmap("Reds") + xlab("layers")
    + scale_y_reverse(
        limits = (-0.5, 52.5), 
        breaks=breaks, labels=custom_labels) 
    + theme(figure_size=(25, 20)) + ylab("") 
    + theme(axis_text_y  = element_text(angle = 0, hjust = 1))
    + ggtitle("Trained Linear Probe")
)
ggsave(
    plot, filename=f"./tutorial_data/pyvene_animal_probe_410m.pdf", dpi=200
)
print(plot)

c:\Users\kjani\AppData\Local\Programs\Python\Python310\lib\site-packages\plotnine\ggplot.py:615: PlotnineWarning: Saving 25 x 20 in image.
c:\Users\kjani\AppData\Local\Programs\Python\Python310\lib\site-packages\plotnine\ggplot.py:616: PlotnineWarning: Filename: ./tutorial_data/pyvene_animal_probe_410m.pdf


<ggplot: (2500 x 2000)>
